In [2]:
import numpy as np
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, random_split, DataLoader
# from torchvision.transforms import ToTensor
from tqdm import trange, tqdm
from matplotlib import pyplot as plt
from model import LSTM
from sklearn.preprocessing import StandardScaler

In [ ]:
device = torch.device(0 if torch.cuda.is_available() else "cpu")
print("Using " + str(device))

# Locate the data
mp_data_path = '/home/prashtata/gradschool/asl/dataset/MP_data'
label_list = os.listdir(mp_data_path) #Since the labels are the directory names, we shall use them

# print(len(label_list))

data = []
labels = []
# num_labels = len(label_list)
num_labels = 300

for i in range(num_labels):

    # List the samples under the label
    label_dir = os.path.join(mp_data_path, label_list[i])
    sample_list = os.listdir(label_dir)

    # Access the samples
    for sample in sample_list:
        kp_file = os.path.join(label_dir, sample, sample+'_keypoints.npy')
        if not os.path.exists(kp_file):
            continue

        keypoints = np.load(kp_file)
        # print(keypoints.shape)

        # Append the keypoints to the data list and the corresponding label to the labels list
        data.append(keypoints); labels.append(i) # Labels will be denoted their index number
        # Add the keypoint augmentations alongside
        data.append(jitter_keypoints(keypoints)); labels.append(i)
        data.append(scale_keypoints(keypoints)); labels.append(i)
        data.append(time_warp_keypoints(keypoints)); labels.append(i)
        data.append(jitter_keypoints(keypoints, noise_level=0.1)); labels.append(i)
        data.append(scale_keypoints(keypoints, scale_range = (0.6, 1.4))); labels.append(i)
        data.append(time_warp_keypoints(keypoints, sigma = 0.4)); labels.append(i)
        

#Standardize the data
scaler = StandardScaler()
data = [scaler.fit_transform(d) for d in data]  # Normalize keypoints

In [ ]:
model_path = "/home/prashtata/gradschool/asl/trained_models/model100_2_02_25_.pth"

model = torch.load(model_path)

In [ ]:
hidden_size = 128
num_stacked_layers = 4

hidden1 = torch.zeros(num_stacked_layers, 1, hidden_size, device=device)
memory1 = torch.zeros(num_stacked_layers, 1, hidden_size, device=device)
pred, hidden1, memory1 = model(input, hidden1, memory1)